# Retail Sales Forecasting Dashboard

This notebook builds a forecasting pipeline for retail sales data and produces outputs ready to import into Power BI. It includes both a Prophet model (if installed) and a SARIMAX fallback, plus a simple sklearn baseline. Replace the sample data section with your real dataset.

**Files produced by this notebook**:
- `sample_retail_sales.csv` (sample data)  
- `forecast.csv` (main model forecast with intervals)  
- `sk_baseline_forecast.csv` (baseline predictions)  

---


In [ ]:
# Imports and plotting settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 12
print('Libraries loaded')

In [ ]:
# === Sample data generation ===
# Replace this block by loading your CSV, e.g. pd.read_csv('your_sales.csv', parse_dates=['ds'])
rng = pd.date_range(start='2019-01-01', end='2022-12-01', freq='MS')  # Monthly start
np.random.seed(42)
trend = np.linspace(200, 500, len(rng))
seasonal = 30 * np.sin(2 * np.pi * (rng.month-1) / 12)
noise = np.random.normal(scale=20, size=len(rng))
sales = trend + seasonal + noise + 50

df = pd.DataFrame({'ds': rng, 'y': sales})
df['month'] = df['ds'].dt.month
df['year'] = df['ds'].dt.year

sample_path = Path('/mnt/data/sample_retail_sales.csv')
df.to_csv(sample_path, index=False)
print(f"Sample data saved to: {sample_path}")

In [ ]:
# === Model: Prophet (if available) or SARIMAX fallback ===
forecast_horizon = 12  # months ahead
future_dates = pd.date_range(start=df['ds'].max() + pd.offsets.MonthBegin(1), periods=forecast_horizon, freq='MS')

use_prophet = False
try:
    from prophet import Prophet
    use_prophet = True
except Exception:
    try:
        from fbprophet import Prophet
        use_prophet = True
    except Exception:
        use_prophet = False

if use_prophet:
    print('Prophet detected. Fitting Prophet model...')
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    m.fit(df[['ds','y']])
    future = m.make_future_dataframe(periods=forecast_horizon, freq='MS')
    forecast = m.predict(future)
    res = forecast[['ds','yhat','yhat_lower','yhat_upper']].set_index('ds')
    merged = df.set_index('ds').join(res, how='outer')
    model_used = 'Prophet'
else:
    print('Prophet not available. Using SARIMAX fallback.')
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    sarimax_order = (1,1,1)
    seasonal_order = (1,1,1,12)
    sarimax_model = SARIMAX(df['y'], order=sarimax_order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
    sarimax_res = sarimax_model.fit(disp=False)
    sarimax_forecast = sarimax_res.get_forecast(steps=forecast_horizon)
    sarimax_index = future_dates
    sarimax_pred = sarimax_forecast.predicted_mean
    sarimax_conf = sarimax_forecast.conf_int()
    res = pd.DataFrame({
        'yhat': sarimax_pred.values,
        'yhat_lower': sarimax_conf.iloc[:,0].values,
        'yhat_upper': sarimax_conf.iloc[:,1].values
    }, index=sarimax_index)
    merged = df.set_index('ds').join(res, how='outer')
    model_used = 'SARIMAX (fallback)'

print('Model used:', model_used)

In [ ]:
# === Save forecast CSV ===
forecast_out = res.reset_index().rename(columns={'index':'ds'})
forecast_csv = Path('/mnt/data/forecast.csv')
forecast_out.to_csv(forecast_csv, index=False)
print('Forecast saved to', forecast_csv)

In [ ]:
# === Plot historical + forecast ===
fig, ax = plt.subplots()
ax.plot(df['ds'], df['y'], label='Historical', marker='o')
ax.plot(res.index, res['yhat'], label='Forecast', linestyle='--', marker='o')
ax.fill_between(res.index, res['yhat_lower'], res['yhat_upper'], alpha=0.25, label='Confidence Interval')
ax.set_title(f'Sales: Historical + {forecast_horizon}-month Forecast ({model_used})')
ax.set_xlabel('Date')
ax.set_ylabel('Sales')
ax.legend()
plt.show()

In [ ]:
# === Decomposition or Prophet components ===
if use_prophet:
    # Prophet components
    from matplotlib import pyplot as plt
    _ = m.plot_components(forecast)
else:
    from statsmodels.tsa.seasonal import seasonal_decompose
    decomposed = seasonal_decompose(df.set_index('ds')['y'], model='additive', period=12)
    fig2, axes = plt.subplots(3,1, figsize=(12,9), sharex=True)
    axes[0].plot(decomposed.trend); axes[0].set_title('Trend')
    axes[1].plot(decomposed.seasonal); axes[1].set_title('Seasonality')
    axes[2].plot(decomposed.resid); axes[2].set_title('Residuals')
    plt.tight_layout()
    plt.show()

In [ ]:
# === SKLearn baseline: LinearRegression with month dummies ===
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

X = df[['month','year']]
y = df['y']

pre = ColumnTransformer([('month_ohe', OneHotEncoder(drop='first', sparse=False), ['month'])], remainder='passthrough')
pipe = make_pipeline(pre, LinearRegression())
pipe.fit(X, y)

future_df = pd.DataFrame({'ds': future_dates})
future_df['month'] = future_df['ds'].dt.month
future_df['year'] = future_df['ds'].dt.year
sk_preds = pipe.predict(future_df[['month','year']])
sk_res = pd.DataFrame({'ds': future_df['ds'], 'sk_yhat': sk_preds})
sk_res.to_csv('/mnt/data/sk_baseline_forecast.csv', index=False)
print('SKLearn baseline saved to /mnt/data/sk_baseline_forecast.csv')

In [ ]:
# === Optional: Simple train-test split evaluation ===
# This runs if dataset is long enough. It splits last 12 months as test.
if len(df) > 24:
    train = df.iloc[:-12].copy()
    test = df.iloc[-12:].copy()
    # Fit simple SARIMAX on train (consistent with earlier fallback)
    try:
        sarimax_model_t = SARIMAX(train['y'], order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
        sarimax_res_t = sarimax_model_t.fit(disp=False)
        pred_test = sarimax_res_t.get_forecast(steps=12).predicted_mean.values
        test['pred'] = pred_test
        from sklearn.metrics import mean_absolute_error, mean_squared_error
        mae = mean_absolute_error(test['y'], test['pred'])
        rmse = mean_squared_error(test['y'], test['pred'], squared=False)
        print(f"Holdout evaluation (last 12 months): MAE={mae:.2f}, RMSE={rmse:.2f}")
    except Exception as e:
        print('Holdout evaluation skipped (error):', e)
else:
    print('Not enough data for a 12-month holdout evaluation.')

In [ ]:
# === Show outputs and where to find files ===
print('Files created:') 
print(' - /mnt/data/sample_retail_sales.csv (replace with your data)')
print(' - /mnt/data/forecast.csv')
print(' - /mnt/data/sk_baseline_forecast.csv')

display(forecast_out.head(12))

---

## Power BI import & visualization quick steps

1. Open Power BI Desktop → **Get Data → Text/CSV** → select `forecast.csv` and `sample_retail_sales.csv` (or your real file).
2. In Power Query: ensure `ds` column is Date type. Create Month/Year columns if needed.
3. Build visuals:
   - **Line chart**: `ds` (X axis) with `y` (actual) and `yhat` (forecast).
   - **Area / ribbon**: Use `yhat_lower` and `yhat_upper` as two series and enable area shading between them, or add a shaded area chart.
   - **Slicers**: Add Year/Month for interactivity.
4. Add KPI cards for last actual, next-month forecast, and error metrics (MAE/RMSE calculated in the notebook).

---

## Next improvements
- Replace sample data with your real dataset (daily/weekly/monthly).  
- Add regressors: promotions, price, holidays, store-level features.  
- Use Prophet changepoints or hyperparameter tuning for SARIMAX.  
- Apply time-series cross-validation (rolling window) to validate model stability.

